In [1]:
import pandas as pd
import numpy as np
from gurobipy import *
import gurobipy as gp
import itertools
import time 

In a local diner, 3 types of jobs required for the diner to run its daily operations: servers, cashiers, cook.

Local diner opens 7 days from 11AM to 10PM for customers, and needs 1 hr before/after to setup/cleanup.

During busy times (lunch hours : 11AM-2PM & dinner hours: 6PM-9PM), the diner needs to run at full-capacity (1 cashiers, 3 servers, 2 cooks). Otherwise, the diner runs at half capacity(1 cashiers, 2 servers, 1 cooks).

Storeowner plans all shifts schedules and wants to ensure 100% coverage of demands **(Rule 1)** to maximize his/her profit. 

There are some rules that the labour union enforces **(Rules 2-7)**: 

* Employees cannot be scheduled for more than 5 days 
* Employees need at least 8 hours between shifts to ensure good rest. 
* Employees cannot work more than 1 shift per day
* Employees cannot work more than 6 shifts per week
* For each day an employee works, the employee must work a minimum of 4 hours and no more than 10 hours
* Employees cannot work less than 20 hours or more than 40 hours per week
* Length of shift must be between 4 and 8 hours










# Set up gurobi model

In [2]:
time_interval = 15
multiplier = int(60/time_interval)

In [3]:
#Gurobi model
m = gp.Model('Toy Example ({} minutes)'.format(time_interval))
m.setParam('Threads', 4)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-11-29
Set parameter Threads to value 4


# Set up parameters and decision variables

In [4]:
lower_index_boolean = False

In [5]:
num_employees = 30
I = num_employees # employee index
J = 7 # day index
K = 24*multiplier # slot index, 12AM to 11PM -> 24 slots
L = 3 # task index

I_list, J_list, K_list, L_list = np.arange(I), np.arange(J), np.arange(K), np.arange(L)

In [6]:
combinations_ijkl = itertools.product(I_list, J_list, K_list, L_list)
combinations_ijkl_list = gp.tuplelist([c for c in combinations_ijkl])

x_ijkl = m.addVars(combinations_ijkl_list.copy(), vtype=GRB.CONTINUOUS, name = 'x_ijkl')
b_ijkl = m.addVars(combinations_ijkl_list.copy(), vtype=GRB.BINARY, name = 'b_ijkl')
e_ijkl = m.addVars(combinations_ijkl_list.copy(), vtype=GRB.BINARY, name = 'e_ijkl')

In [7]:
combinations_jkl = itertools.product(J_list, K_list, L_list)
combinations_jkl_list = gp.tuplelist([c for c in combinations_jkl])

su_jkl = m.addVars(combinations_jkl_list.copy(), vtype=GRB.CONTINUOUS, name = 'su_jkl')
so_jkl = m.addVars(combinations_jkl_list.copy(), vtype=GRB.CONTINUOUS, name = 'so_jkl')

In [8]:
combinations_ij = itertools.product(I_list, J_list)
combinations_ij_list = gp.tuplelist([c for c in combinations_ij])

W_ij = m.addVars(combinations_ij_list.copy(), vtype=GRB.BINARY, name = 'W_ij')

# Specify data

In [9]:
# Data: Demand
d1 = np.repeat(np.array([[1,2,1]]), 11*multiplier, axis=0) # 0, ..., 10
d2 = np.repeat(np.array([[1,3,2]]), 4*multiplier, axis=0) # 11, ..., 14 (busy)
d3 = np.repeat(np.array([[1,2,1]]), 3*multiplier, axis=0) # 15, ..., 17
d4 = np.repeat(np.array([[1,3,2]]), 4*multiplier, axis=0) # 18, ..., 21 (busy)
d5 = np.repeat(np.array([[1,2,1]]), 2*multiplier, axis=0) # 22, 23
demand = np.concatenate((d1, d2, d3, d4, d5), axis=0)

d_jkl = {}
for j in range(J):
    for k in range(K):
        for l in range(L):
            d_jkl[(j,k,l)] = demand[k,l]
d_jkl_combo, d_jkl = gp.multidict(d_jkl)

In [10]:
# Data: Employee time availability 
time_avail_10emp = np.array([[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0],
                      [0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0],
                      [0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0],
                      [0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
                      [0,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,1],
                      [1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
                      [1,1,1,1,1,1,1,1,1,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1],
                      [0,0,0,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,1,1],
                      [1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0],
                      [1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1]])
time_avail_10emp = np.repeat(time_avail_10emp, multiplier, axis=1)
time_avail = np.repeat(time_avail_10emp, 3, axis=0)
new_col = [str(i) for i in range(time_avail.shape[1])]
df_time_avail= pd.DataFrame(data = time_avail, columns=new_col, index=[i for i in np.arange(0,num_employees)] )

In [11]:
# Data: Employee skills 
employee_skills_10emp = np.array([[0,1,1],[1,1,0],[0,1,0],
                            [0,1,0],[0,0,1],[1,1,1],
                            [1,0,1],[1,1,0],[0,1,0],[0,1,1]])
employee_skills = np.repeat(employee_skills_10emp, 3, axis = 0)

df_employee_skills = pd.DataFrame(columns=['cashier', 'server', 'cook'], data=employee_skills,  index=[i for i in np.arange(0,num_employees)])
df_cashier, df_server, df_cook = df_time_avail.copy(), df_time_avail.copy(), df_time_avail.copy()

for employee in df_employee_skills.index:
    if df_employee_skills.loc[employee, 'cashier'] == 0:
        df_cashier.loc[employee] = np.zeros(df_time_avail.shape[1],)
    if df_employee_skills.loc[employee, 'server'] == 0:
        df_server.loc[employee] = np.zeros(df_time_avail.shape[1],)
    if df_employee_skills.loc[employee, 'cook'] == 0:
        df_cook.loc[employee] = np.zeros(df_time_avail.shape[1],)

In [12]:
y_ijkl = {} # time & skill availability
df_mapping = {0: df_cashier, 1: df_server, 2: df_cook} # 0 => 'cashier' 1 => 'front_server' => 'cook'
for i in range(I):
    for j in range(J):
        for k in range(K):
            for l in range(L):
                y_ijkl[(i,j,k,l)] = df_mapping[l].loc[i,str(k)]

y_ijkl_combo, y_ijkl = gp.multidict(y_ijkl)

In [13]:
'''
ASSUMPTION:
1. Equal penalties for any amount of overstaffing and understaffing 
    (e.g., 1 employee short is treated the same way as 2 employees short)
2. Busy hours -- lunch hours : 11-14; dinner hours: 18-21
3. Same set of penalties applied to all days

OVERSTAFFING:
1. cashier penalty: busy hours = 1, otherwise = 2
2. server penalty: busy hours = 2, o/w = 3
3. cook penalty: busy hours = 1, o/w = 2

UNDERSTAFFING:
1. cashier penalty: busy hours = 5, otherwise = 2
2. server penalty: busy hours = 3, o/w = 1
3. cook penalty: busy hours = 4, o/w = 2

'''
# [cashier, server, cook]
o1 = np.repeat(np.array([[2,3,2]]), 11*multiplier, axis=0) # 0, ..., 10
o2 = np.repeat(np.array([[1,2,1]]), 4*multiplier, axis=0) # 11, ..., 14 (busy)
o3 = np.repeat(np.array([[2,3,2]]), 3*multiplier, axis=0) # 15, ..., 17
o4 = np.repeat(np.array([[1,2,1]]), 4*multiplier, axis=0) # 18, ..., 21 (busy)
o5 = np.repeat(np.array([[2,3,2]]), 2*multiplier, axis=0) # 22, 23
overstaffing = np.concatenate((o1, o2, o3, o4, o5), axis=0)

u1 = np.repeat(np.array([[2,1,2]]), 11*multiplier, axis=0) # 0, ..., 10
u2 = np.repeat(np.array([[5,3,4]]), 4*multiplier, axis=0) # 11, ..., 14 (busy)
u3 = np.repeat(np.array([[2,1,2]]), 3*multiplier, axis=0) # 15, ..., 17
u4 = np.repeat(np.array([[5,3,4]]), 4*multiplier, axis=0) # 18, ..., 21 (busy)
u5 = np.repeat(np.array([[2,1,2]]), 2*multiplier, axis=0) # 22, 23
understaffing = np.concatenate((u1, u2, u3, u4, u5), axis=0)

po_jkl = {} # overstaffing penalty
pu_jkl = {} # understaffing penalty

for j in range(J):
    for k in range(K):
        for l in range(L):
            po_jkl[(j,k,l)] = overstaffing[k,l]
            pu_jkl[(j,k,l)] = understaffing[k,l]

po_jkl_combo, po_jkl = gp.multidict(po_jkl)
pu_jkl_combo, pu_jkl = gp.multidict(pu_jkl)

# Objective function

In [14]:
m.setObjective(so_jkl.prod(po_jkl)+su_jkl.prod(pu_jkl), GRB.MINIMIZE)

# Constraints:

1. Ensure 100% Demand Coverage

In [15]:
coverage_level = 1.0
m.addConstrs((x_ijkl.sum('*', j, k, l) >= coverage_level * d_jkl[j,k,l] for j in range(J) for k in range(K) for l in range(L)), name='coverage')

{(0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6, 0): <gurobi.Constr *Awaiting Model Upd

2. Employees cannot be scheduled for more than 5 days

In [16]:
max_num_days_scheduled = 5
# number 5, 6 and 7
for i in range(I):
    for j in range(6,J+1):
        W_ij_sum = 0
        for w in range(j-6, j):
            W_ij_sum += W_ij[i,w]
        m.addConstr(W_ij_sum <= max_num_days_scheduled, name='const5_{}{}'.format(i, j)) #5

for i in range(I):
    for j in range(J):
        m.addConstr(x_ijkl.sum(i, j, '*', '*') >= W_ij[i,j], name='const6_{}{}'.format(i, j)) #6
        m.addConstr(x_ijkl.sum(i, j, '*', '*') <= 9999*W_ij[i,j], name='const7_{}{}'.format(i, j)) #7

3. Employees need at least 8 hours between shifts to ensure good rest

In [17]:
# number 8 and 9
for i in range(I):
    for j in range(J):
        for k in range(K):
            if k<16*multiplier:
# For this toy example, k is indexed differently
                m.addConstr((e_ijkl.sum(i,j,k,'*') + b_ijkl.sum(i,j,range(k,k+8*multiplier),'*') <= 1), name='const8_{}{}{}'.format(i,j,k))
            elif k >=16*multiplier and j < J-1:
                m.addConstr((e_ijkl.sum(i,j,k,'*') + b_ijkl.sum(i,j,range(k,K),'*') + b_ijkl.sum(i,j+1,range(k-16*multiplier),'*') <= 1), name='const9_{}{}{}'.format(i,j,k))

4. Employees cannot work more than 1 shift per day

In [18]:
max_shift_per_day = 1
m.addConstrs((b_ijkl.sum(i, j, '*', '*') <= max_shift_per_day for i in range(I) for j in range(J)), name='max_shift_per_day')

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,


5. Employees cannot work more than 6 shifts per week

In [19]:
max_shift_per_week = 6
m.addConstrs((b_ijkl.sum(i, '*', '*', '*') <= max_shift_per_week for i in range(I)), name='max_shift_per_week')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

6. For each day an employee works, the employee must work a minimum of 4 hours and no more than 10 hours

In [20]:
min_daily_hrs, max_daily_hrs = 4, 10
m.addConstrs((x_ijkl.sum(i, j, '*', '*') >= min_daily_hrs*multiplier*W_ij[i,j] for i in range(I) for j in range(J)), name='min_daily_hours')
m.addConstrs((x_ijkl.sum(i, j, '*', '*') <= max_daily_hrs*multiplier*W_ij[i,j] for i in range(I) for j in range(J)), name='max_daily_hours')

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 6): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,


7. Employees cannot work less than 20 hours or more than 40 hours per week

In [21]:
min_weekly_hrs, max_weekly_hrs = 20, 40
m.addConstrs((x_ijkl.sum(i, '*', '*', '*') >= min_weekly_hrs*multiplier for i in range(I)), name='min_weekly_hours')
m.addConstrs((x_ijkl.sum(i, '*', '*', '*') <= max_weekly_hrs*multiplier for i in range(I)), name='max_weekly_hours')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

8. Length of shift must be between 4 hours and 8 hours

In [22]:
# number 16-17
for i in range(I):
    for j in range(J):
        for k in range(K):
            if k<20*multiplier:
# For this toy example, k is indexed differently
                m.addConstr((b_ijkl.sum(i,j,k,'*') + e_ijkl.sum(i,j,range(k,k+4*multiplier),'*') <= 1), name='const16_{}{}{}'.format(i,j,k))
            elif k >= 20*multiplier and j < J-1:
                m.addConstr((b_ijkl.sum(i,j,k,'*') + e_ijkl.sum(i,j,range(k,K),'*') + e_ijkl.sum(i,j+1,range(k-20*multiplier),'*') <= 1), name='const17_{}{}{}'.format(i,j,k))
            elif k >= 20*multiplier and j == J-1:
                m.addConstr((b_ijkl.sum(i,j,k,'*') + 1 <= 1), name='const16_{}{}{}_complimentary'.format(i,j,k))

In [23]:
# new: number 18-19
for i in range(I):
    for j in range(J):
        for k in range(K):
            for l in range(L):
                if k<16*multiplier:
                    m.addConstr((e_ijkl.sum(i,j,range(k,k+8*multiplier),l) >= b_ijkl[i,j,k,l]), 
                                name='const18_{}{}{}'.format(i,j,k))
                elif k >=16*multiplier and j < J-1:
                    m.addConstr((e_ijkl.sum(i,j,range(k,K),l) + e_ijkl.sum(i,j+1,range(k+8*multiplier-24*multiplier),l) >= b_ijkl[i,j,k,l]), 
                                name='const19_{}{}{}'.format(i,j,k))

# Other Constraints  

Employee cannot be scheduled if not available

In [24]:
m.addConstrs((x_ijkl[i,j,k,l] <= y_ijkl[i,j,k,l] for i in range(I) for j in range(J) for k in range(K) for l in range(L)), name='availability')

{(0, 0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 5, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 5, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 5, 2): <gurobi.Constr *Awaiting Model Up

Demand Balance 

In [25]:
m.addConstrs((x_ijkl.sum('*', j, k, l) - so_jkl[j,k,l] + su_jkl[j,k,l] == d_jkl[j,k,l] for j in range(J) for k in range(K) for l in range(L)), name='demand_balance')

{(0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6, 0): <gurobi.Constr *Awaiting Model Upd

An employee can be assigned to one type of task l on day j and slot k

In [26]:
m.addConstrs((x_ijkl.sum(i, j, k, '*') <= 1 for i in range(I) for j in range(J) for k in range(K)), name='one_task')

{(0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 18): <gurobi.Constr *Awaiting 

Relationship between x, b, and e

In [27]:
# number 20-25
m.addConstrs((b_ijkl[i,j,k,l] <= x_ijkl[i,j,k,l] for i in range(I) for j in range(J) for k in range(K) for l in range(L)), name='const20')
m.addConstrs((e_ijkl[i,j,k,l] <= (1 - x_ijkl[i,j,k,l]) for i in range(I) for j in range(J) for k in range(K) for l in range(L)), name='const24')

m.addConstrs((b_ijkl[i,j,k,l] <= (1 - x_ijkl[i,j,k-1,l]) for i in range(I) for j in range(J) for k in range(1,K) for l in range(L)), name='const21')
m.addConstrs((x_ijkl[i,j,k-1,l] + b_ijkl[i,j,k,l] >= x_ijkl[i,j,k,l] for i in range(I) for j in range(J) for k in range(1,K) for l in range(L)), name='const22')
m.addConstrs((e_ijkl[i,j,k,l] <= x_ijkl[i,j,k-1,l] for i in range(I) for j in range(J) for k in range(1,K) for l in range(L)), name='const23')
m.addConstrs((x_ijkl[i,j,k-1,l] - x_ijkl[i,j,k,l] <= e_ijkl[i,j,k,l] for i in range(I) for j in range(J) for k in range(1,K) for l in range(L)), name='const25')

{(0, 0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 5, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 5, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 5, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 6, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 6, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 6, 2): <gurobi.Constr *Awaiting Model Up

In [28]:
# number 26-29
m.addConstrs((b_ijkl[i,j+1,0,l] <= (1 - x_ijkl[i,j,K-1,l]) for i in range(I) for j in range(J-1) for l in range(L)), name='const26')
m.addConstrs((x_ijkl[i,j,K-1,l] + b_ijkl[i,j+1,0,l] >= x_ijkl[i,j+1,0,l] for i in range(I) for j in range(J-1) for l in range(L)), name='const27')

m.addConstrs((e_ijkl[i,j+1,0,l] <= x_ijkl[i,j,K-1,l] for i in range(I) for j in range(J-1) for l in range(L)), name='const28')
m.addConstrs((x_ijkl[i,j,K-1,l] - x_ijkl[i,j+1,0,l] <= e_ijkl[i,j+1,0,l] for i in range(I) for j in range(J-1) for l in range(L)), name='const29')

{(0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0, 0): <gurobi.Constr *Awaiting Model Upd

In [29]:
m.addConstrs((b_ijkl[i,0,0,l] == x_ijkl[i,0,0,l] for i in range(I) for l in range(L)), name = 'const22_complementary')
# m.addConstrs((e_ijkl[i,6,23,l] >= x_ijkl[i,6,23,l] for i in range(I) for l in range(L)), name='const22_complementary')

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (5, 0): <gurobi.Constr *Awaiting Model Update*>,
 (5, 1): <gurobi.Constr *Awaiting Model Update*>,
 (5, 2): <gurobi.Constr *Awaiting Model Update*>,
 (6, 0): <gurobi.Constr *Awaiting Model Update*>,
 (6, 1): <gurobi.Constr *Awaiting Model Update*>,


Lower Index First

In [30]:
if lower_index_boolean == True:
    for i in range(I-1):
        for j in range(J):
            m.addConstr((7*W_ij.sum(i,range(0,j)) >= W_ij.sum(i+1,range(0,j))), name= 'lower index 1')

    for i in range(I): # the first I-1 employee
        for i_prime in range(i+1, I):
            for l in range(L):
                for j in range(J): 
                    checker = True
                    for k in range(K):
                        if y_ijkl[i,j,k,l] == y_ijkl[i_prime,j,k,l]:
                            checker = True
                        else:
                            checker = False
                            break
                    if checker == True: # for i and i_prime on task l and day j, they have the same availability
                        # assign lower index first
                        for k in range(K):
                            m.addConstr((x_ijkl[i,j,k,l] + W_ij[i,j]  >= x_ijkl[i_prime,j,k,l]), name = 'lower index 2')

Save the model for inspection

In [31]:
m.write('Toy_Example_{}Employees_{}min_interval.lp'.format(num_employees, time_interval))

Solve

In [32]:
s = time.time()
m.optimize()
print(time.time()-s)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 545442 rows, 185682 columns and 6383712 nonzeros
Model fingerprint: 0x754a4183
Variable types: 64512 continuous, 121170 integer (121170 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 330090 rows and 108396 columns (presolve time = 5s) ...
Presolve removed 387660 rows and 109481 columns (presolve time = 10s) ...
Presolve removed 387783 rows and 109511 columns (presolve time = 17s) ...
Presolve removed 398356 rows and 121654 columns (presolve time = 20s) ...
Presolve removed 405421 rows and 121802 columns (presolve time = 25s) ...
Presolve removed 403526 rows and 119738 columns
Presolve time: 28.54s
Presolved: 141916 rows, 65944 columns, 3272195 nonzeros
Variable types: 24318 continuous, 41626 int

H 5372  5015                      91.0000000    0.00000   100%  1914 31519s
H 5372  4764                      80.0000000    0.00000   100%  1914 31519s
H 5372  4525                       6.0000000    0.00000   100%  1914 31519s
H 5372  4299                       0.0000000    0.00000  0.00%  1914 31519s

Cutting planes:
  Gomory: 4
  Cover: 6
  Projected implied bound: 5
  Clique: 11
  MIR: 20
  StrongCG: 2
  Flow cover: 11
  GUB cover: 3
  Zero half: 240
  Relax-and-lift: 14

Explored 5372 nodes (12021800 simplex iterations) in 31560.63 seconds (32957.84 work units)
Thread count was 4 (of 4 available processors)

Solution count 4: 0 6 80 91 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
31560.70514702797


In [33]:
# for i in range(I):
#     for l in range(L):
#         print(e_ijkl[(i,6,23,l)].X , x_ijkl[(i,6,23,l)].X)

In [34]:
# for i in range(I):
#     for l in range(L):
#         print(b_ijkl[(i,0,0,l)].X , x_ijkl[(i,0,0,l)].X)

In [35]:
# m.computeIIS()
# m.write("model.ilp")

In [36]:
# # Display optimal values of decision variables
# for v in m.getVars():
#     if v.x > 1e-6:
#         print(v.varName, v.x)

# # Display optimal total penalty
# print('Total penalty: ', m.objVal)

In [37]:
import openpyxl

In [38]:
def write_to_excel_xbe(excel_name, dec_var_dict):
    # x_ijkl, b_ijkl, e_ijkl
    for j in range(J):
        with pd.ExcelWriter('Day_{}_{}.xlsx'.format(j+1, excel_name)) as writer:  
            for i in range(I):
                df_results = pd.DataFrame(data = np.zeros((K,L)), columns = ['cashier', 'server', 'cook'], index = new_col)
                for k in range(K):
                    for l in range(L):
                        df_results.iloc[k,l] = dec_var_dict[(i,j,k,l)]
                df_results.to_excel(writer, sheet_name='Employee{}'.format(i+1))

In [39]:
def write_to_excel_s(excel_name, dec_var_dict):
    # understaffing (su_jkl), overstaffing (so_jkl)
    with pd.ExcelWriter('{}_sjkl.xlsx'.format(excel_name)) as writer:  
        for j in range(J):
            df_results = pd.DataFrame(data = np.zeros((K,L)), columns = ['cashier', 'server', 'cook'], index = new_col)
            for k in range(K):
                for l in range(L):
                    df_results.iloc[k,l] = dec_var_dict[(j,k,l)]

            df_results.to_excel(writer, sheet_name='Day_{}'.format(j+1))
        writer.save()

In [40]:
x_ijkl_dict = {k : v.X for k,v in x_ijkl.items()}
b_ijkl_dict = {k : v.X for k,v in b_ijkl.items()}
e_ijkl_dict = {k : v.X for k,v in e_ijkl.items()}

write_to_excel_xbe(excel_name='x_ijkl', dec_var_dict=x_ijkl_dict)
write_to_excel_xbe(excel_name='b_ijkl', dec_var_dict=b_ijkl_dict)
write_to_excel_xbe(excel_name='e_ijkl', dec_var_dict=e_ijkl_dict)

In [41]:
su_jkl_dict = {k : v.X for k,v in su_jkl.items()}
so_jkl_dict = {k : v.X for k,v in so_jkl.items()}

write_to_excel_s(excel_name='understaffing', dec_var_dict=su_jkl_dict)
write_to_excel_s(excel_name='overstaffing', dec_var_dict=so_jkl_dict)

In [42]:
W_ij_dict = {k : v.X for k,v in W_ij.items()}
df_W_ij = pd.DataFrame(data = np.zeros((I,J)), columns = [j for j in np.arange(J)], index=[i for i in np.arange(0,num_employees)])
with pd.ExcelWriter('W_ij.xlsx') as writer:  
    for i in range(I):
        for j in range(J):
            df_W_ij.iloc[i,j] = W_ij[(i,j)]
    
    df_W_ij.to_excel(writer, sheet_name = 'W_ij')

In [43]:
occupation = ['cashier', 'server', 'cook']
schedule = pd.DataFrame(data = np.zeros((I,J)), columns = ["Day{}".format(j+1) for j in np.arange(J)], 
                            index=['Employee{}'.format(i) for i in np.arange(0,I)],
                            dtype = np.str)
for i in range(I):
    start = []
    end = []
    for j in range(J):
        for k in range(K):
            for l in range(L):
                if b_ijkl_dict[(i,j,k,l)] == 1:
                    start.append([j,k,l])
                if e_ijkl_dict[(i,j,k,l)] == 1:
                    end.append([j,k,l])
    if len(start)!= len(end):
        end.append([6,24,start[-1][2]])
    for index in range(len(start)):
        start_time = start[index][1]
        end_time = end[index][1]
        o_index = start[index][2]
        day = end[index][0]
        if start_time > end_time:
            if schedule.iloc[i,day-1] == "0.0":
                schedule.iloc[i,day-1] = ''
                schedule.iloc[i,day-1] += '{} - tomorrow {} as {}'.format(start_time, end_time, occupation[o_index])
            else:
                schedule.iloc[i,day-1] += '{} - tomorrow {} as {}'.format(start_time, end_time, occupation[o_index])
        else:
            if schedule.iloc[i,day] == "0.0":
                schedule.iloc[i,day] = ''
            schedule.iloc[i,day] += '{} - {} as {}'.format(start_time, end_time, occupation[o_index])
            #schedule.iloc[i,day] += '{} -{} ({})'.format(start_time,end_time,occupation[o_index])
schedule.to_excel("updated_schedule_save.xlsx")
schedule

/Users/lsjnancy/opt/anaconda3/envs/mie490_capstone/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


,Day1,Day2,Day3,Day4,Day5,Day6,Day7
Employee0,0.0,44 - 60 as cook,44 - 60 as server,0.0,44 - 60 as cook,16 - 34 as server,44 - 60 as cook
Employee1,44 - 60 as cook,0.0,44 - 60 as cook,44 - 60 as cook,0 - 23 as server,0.0,0 - 16 as cook
Employee2,0.0,0 - 29 as cook,0.0,0 - 30 as cook,24 - 48 as cook,41 - 59 as server,19 - 35 as server
Employee3,49 - 72 as server,36 - 61 as cashier,20 - 42 as server,37 - 60 as cashier,46 - 71 as server,0.0,30 - 46 as cashier
Employee4,34 - 50 as cashier,0.0,44 - 73 as cashier,12 - 37 as cashier,23 - 46 as server,10 - 40 as cashier,0.0
Employee5,16 - 42 as server,18 - 41 as server,0.0,57 - 80 as server,28 - 44 as cashier,0.0,0.0
Employee6,72 - 88 as server,71 - 88 as server,0.0,54 - 76 as server,44 - 60 as server,19 - 41 as server,20 - 39 as server
Employee7,44 - 60 as server,0.0,19 - 41 as server,18 - 37 as server,16 - 42 as server,0.0,44 - 60 as server
Employee8,17 - 33 as server,41 - 71 as server,72 - 88 as server,44 - 60 as server,0.0,44 - 72 as server,57 - 88 as server
Employee9,72 - tomorrow 0 as server,49 - 79 as server,78 - tomorrow 0 as server,0.0,42 - 60 as server,59 - 76 as server79 - tomorrow 24 as server,0.0


In [44]:
# for i in range(I):
#     for j in range(J):
#         print(W_ij[(i,j)])

In [45]:
# for j in range(J):
#     for k in range(K):
#         for l in range(L):
#             if b_ijkl[(15,j,k,l)].X  == 1:
#                 print("#################bbbbbbbb################")
#                 print(j,k,l)
#                 print("#################bbbbbbbb################")
#             if x_ijkl[(15,j,k,l)].X  == 1:
#                 print(j,k,l)
#             if e_ijkl[(15,j,k,l)].X  == 1:
#                 print("#################eeeeeee################")
#                 print(j,k,l)
#                 print("#################eeeeeee################")

In [46]:
# print('Day 5')
# for k in range(K):
#     for l in range(L):
#         if x_ijkl_dict[(24, 4, k, l)] > 0:
#             print("hour {}, task {}: {}".format(k, l, x_ijkl_dict[(24, 4, k, l)]))

# print('Day 6')
# for k in range(K):
#     for l in range(L):
#         if x_ijkl_dict[(24, 5, k, l)] > 0:
#             print("hour {}, task {}: {}".format(k, l, x_ijkl_dict[(24, 5, k, l)]))

In [47]:
# for v in x_ijkl.values():
#     print("{} : {}".format(v.varName, v.x))